In [2]:
import pandas as pd
df_a = pd.read_csv(r'G:\AI\deep_learning\TaskAI\data\dataset_a.csv')
df_b = pd.read_csv(r'G:\AI\deep_learning\TaskAI\data\dataset_b.csv')

In [3]:
df_a.head()

,ID_CRM,Company_Name,City,Industry
0,1,UzAuto Motors,Andijan,Manufacturing
1,2,Artel Electronics,Tashkent,Electronics
2,3,Akfa Group,Tashkent,Construction
3,4,Tashkent IT Park,Tashkent,IT
4,5,Korzinka.uz,Tashkent,Retail


In [4]:
df_b.head()

,ID_WEB,Org_Name,Location,Description
0,1001,Uz-Auto Motors AJ,Andijon vil.,Avtomobil ishlab chiqarish
1,1002,Artel Elektroniks,Toshkent sh.,Maishiy texnika
2,1003,Akfa Holding,Tashkent,Qurilish materiallari va derazalar
3,1004,IT-Park Toshkent,"Toshkent, Mirzo Ulugbek",IT startaplar markazi
4,1005,Korzinka Supermarket,Toshkent,Oziq-ovqat savdosi


In [6]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer('all-MiniLM-L6-v2')


list_a = (df_a['Company_Name'] + " in " + df_a['City'] + " doing " + df_a['Industry']).tolist()
list_b = (df_b['Org_Name'] + " in " + df_b['Location'] + " description: " + df_b['Description']).tolist()

print("AI ma'lumotlarni tahlil qilmoqda (embedding yaratmoqda)...")

embeddings_a = model.encode(list_a, convert_to_tensor=True)
embeddings_b = model.encode(list_b, convert_to_tensor=True)

cosine_scores = util.cos_sim(embeddings_a, embeddings_b)

matches = []
threshold = 0.5  # O'xshashlik darajasi 50% dan yuqori bo'lganlarni olamiz

for i in range(len(list_a)):
    score, index = torch.max(cosine_scores[i], dim=0)
    
    if score.item() > threshold:
        matches.append({
            'ID_CRM': df_a.iloc[i]['ID_CRM'],
            'Company_A': df_a.iloc[i]['Company_Name'],
            'Match_Score': round(score.item(), 3),
            'Company_B': df_b.iloc[index.item()]['Org_Name'],
            'ID_WEB': df_b.iloc[index.item()]['ID_WEB']
        })

df_result = pd.DataFrame(matches)

df_result.to_csv('matched_datasets.csv', index=False)
print("\nBirlashtirish yakunlandi! Dastlabki 10 ta natija:")
print(df_result.head(10))

AI ma'lumotlarni tahlil qilmoqda (embedding yaratmoqda)...

Birlashtirish yakunlandi! Dastlabki 10 ta natija:
   ID_CRM          Company_A  Match_Score             Company_B  ID_WEB
0       1      UzAuto Motors        0.656     Uz-Auto Motors AJ    1001
1       2  Artel Electronics        0.504     Artel Elektroniks    1002
2       3         Akfa Group        0.650          Akfa Holding    1003
3       4   Tashkent IT Park        0.599      IT-Park Toshkent    1004
4       5        Korzinka.uz        0.683  Korzinka Supermarket    1005
5       6         Hamkorbank        0.823           Hamkor Bank    1006
6       7        Kapitalbank        0.763      Kapital Bank ATB    1007
7       8           SQB Bank        0.668  Sanoat Qurilish Bank    1008
8      12        BMB Holding        0.549       BMB Trade Group    1012
9      13       Golden House        0.673          Golden-House    1013
